In [2]:
import numpy
import pandas

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [3]:
# load dataset
dataframe = pandas.read_csv("../data/OnlineNewsPopularity.csv")
continuous_dataframe=dataframe.iloc[:,numpy.r_[2:13, 19:31, 39:61]]
target_dataframe=dataframe.iloc[:,60]
boolean_dataframe=dataframe.iloc[:,numpy.r_[13:19, 31:39]]
continuous_dataframe = (continuous_dataframe - continuous_dataframe.min()) / (continuous_dataframe.max() - continuous_dataframe.min())
target_dataframe=numpy.log10(target_dataframe)

dataset=pandas.concat([continuous_dataframe,boolean_dataframe],axis=1)
dataset = dataset.values

# split into input (X) and output (Y) variables
X = dataset[:,0:-1]
Y = target_dataframe.values
test1=pandas.concat([target_dataframe,10**target_dataframe],axis=1)


In [4]:
# define custom loss function
import keras
def root_mean_squared_error(y_true, y_pred):
    #print("True:"+(10**y_true)+"   predicted:"+(10**y_pred))
    #return keras.backend.sqrt(keras.backend.mean(keras.backend.square(y_pred - y_true),axis=-1)) 
    return (y_pred - y_true)

# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(59, input_dim=58, kernel_initializer='normal', activation='relu'))
    model.add(Dense(25, kernel_initializer='normal', activation='relu'))
    model.add(Dense(12, kernel_initializer='normal', activation='relu'))
    model.add(Dense(6, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss="mean_absolute_percentage_error", optimizer='adam')
    return model

In [6]:
# fix random seed for reproducibility
seed = 7
# evaluate model with standardized dataset
numpy.random.seed(seed)
estimators = []
#estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=4, batch_size=5, verbose=1)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=4, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
#results = 10 ** results
print("Standardized: %.2f (%.2f) RMSE" % (results.mean(), results.std()))

Epoch 1/4
21445/29733 [====================>.........] - ETA: 2s - loss: 152219.0134

KeyboardInterrupt: 

In [ ]:
print(results)